In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

In [6]:
def sam_analysis():
    print('=======================================')

    print()
    print('사회계정행렬 선택: ')
    print('1. 거시 사회계정행렬 \n 2. 거시 사회계정행렬(국산) \n 3. 미시 사회계정행렬(산업 세분화) \n 4. 미시 사회계정행렬(산업 세분화: 국산) \n 5. 미시 사회계정행렬(최종) \n 6. 미시 사회계정행렬(최종: 국산) \n 7. 가계, 노동 분할 미시 사회계정행렬(최종:국산)')
    print()
    i = int(input('승수 분석을 진행할 SAM의 종류를 선택해주세요. : '))
    
    
    if i == 1:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM.xlsx')
        
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM.xlsx')
        print('데이터 불러오기 완료')
        print('========================================')
        name = list(data.columns[6:])
    elif i == 2:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM(국산).xlsx')
         
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM(국산).xlsx')
        print('데이터 불러오기 완료')
        print('========================================')
        name = list(data.columns[6:])
    elif i == 3:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬 RAS 적용.xlsx')
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬 RAS 적용.xlsx')
        name = list(data.columns[26:])
    elif i == 4:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(국산) RAS 적용.xlsx')
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(국산) RAS 적용.xlsx')
        name = list(data.columns[26:])
    
    elif i == 5:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종) RAS 적용.xlsx')        
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종) RAS 적용.xlsx')        
        name = list(data.columns[27:])
    elif i == 6:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종 국산) RAS 적용.xlsx')
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종 국산) RAS 적용.xlsx')
        name = list(data.columns[27:])
        
    
    elif i == 7:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/가계, 노동 미시 버전 사회계정행렬(국산) RAS 적용.xlsx')
        
        # window
        data = pd.read_excel('/Users/dongyokim/main/jupyter/공간정보프로젝트/RAS 전처리 SAM/승수분석용 데이터/가계, 노동 미시 버전 사회계정행렬(국산) RAS 적용.xlsx')
        name = list(data.columns[32:])
    else:
        print('번호를 잘못 입력했습니다. 함수를 다시 실행해주세요. ')
        print('========================================')
    
    # untitled:0 삭제
    del name[0]
    
    # 총계 삭제
    del name[-1]
    data.rename(columns={'Unnamed: 0':'구분'},inplace=True)
    data.set_index('구분', inplace=True)
    data.fillna(0, inplace=True)
    data.replace(' ', 0, inplace=True)
    data = data.astype('float')
    
    
    print()
    
    print('데이터 전처리 및 불러오기 완료')
    print('========================================')
    
    
    s11 = data.loc[:'정부', :'정부'].values.copy()

    s12 = data.loc[:'정부', '고정자본형성':'수입'].values.copy()
    
    s21 = data.loc['고정자본형성':'수입', :'정부'].values.copy()
    
    s22 = data.loc['고정자본형성':'수입', '고정자본형성':'수입'].values.copy()
    
    
    y1_r = data.loc['총계', :'정부'].values.reshape(1, -1).copy()
    
    y2_r = data.loc['총계', '고정자본형성':'수입'].values.reshape(1, -1).copy()
        

    y1_c = data.loc[:'정부', '총계'].values.reshape(-1, 1).copy()
    y2_c = data.loc['고정자본형성':'수입', '총계'].values.reshape(-1, 1).copy()
    
    
    # 투입계수 도출 
    
    h11 = s11 / y1_r
    h12 = s12 / y2_r
    h21 = s21 / y1_r
    h22 = s22 / y2_r
    
        
    # 승수행렬
    SAM_multiplier = np.linalg.inv(np.identity(h11.shape[0])-h11)   
    print()
    print('승수 행렬 : ')
    print(SAM_multiplier)
    print()
    
    print(name)
    print()
    added_name = input("값을 추가하고 싶은 계정 이름을 입력해주세요: ")
    
    name_index = name.index(added_name)
    
    print('=======================')
    added_value = float(input('추가하실 금액을 입력해주세요: '))
    
    standard_value = y2_r[0][name_index].copy()
    
    y2_r[0][name_index] = y2_r[0][name_index] + added_value
    
    
    print("{}({})항목에 + {} = {}".format(added_name, standard_value,added_value, y2_r[0][name_index]))
    
    result_vector = np.dot(np.dot(SAM_multiplier, h12), y2_r.transpose()).transpose()    

    update_metrix = h11 * result_vector    
    
    
    data.loc[:'정부', :'정부'] = update_metrix

    data.drop(columns=['총계'], index=['총계'], inplace=True)
    
    
    print()
    print('승수분석을 완료하였습니다. ')
    print('=======================')
    display(data)
    
    return SAM_multiplier, data

In [8]:
sam_multiplier, data = sam_analysis()


사회계정행렬 선택: 
1. 거시 사회계정행렬 
 2. 거시 사회계정행렬(국산) 
 3. 미시 사회계정행렬(산업 세분화) 
 4. 미시 사회계정행렬(산업 세분화: 국산) 
 5. 미시 사회계정행렬(최종) 
 6. 미시 사회계정행렬(최종: 국산) 
 7. 가계, 노동 분할 미시 사회계정행렬(최종:국산)

승수 분석을 진행할 SAM의 종류를 선택해주세요. : 6

데이터 전처리 및 불러오기 완료

승수 행렬 : 
[[1.09453295e+00 1.10878913e-03 4.69765337e-02 3.95682346e-02
  2.49526445e-02 4.40411420e-02 3.36882569e-02 9.16921590e-02
  3.38171618e-02 3.58062853e-02 2.75710098e-02 3.87813489e-02
  4.35087420e-02 4.84907614e-02 5.13019479e-02 4.12111882e-02
  2.45868113e-02 3.63117429e-02 2.75988960e-02 3.37697158e-02
  4.84687496e-02 5.35051986e-02 2.22576171e-02 6.45586335e-03
  5.35051986e-02 6.45586335e-03 3.33382533e-02]
 [3.82749153e-02 1.00159211e+00 8.55543910e-02 3.82217995e-02
  1.57696641e-01 7.19658203e-02 4.08807826e-02 4.83553104e-02
  3.28726235e-02 3.19121475e-02 2.78474188e-02 4.01617760e-02
  3.87583110e-02 4.42371741e-02 5.01587933e-02 4.55924778e-02
  3.37757988e-02 3.57144633e-02 2.58614806e-02 3.00787823e-02
  3.95758388e-02 4.22397558e-02 1.81939

값을 추가하고 싶은 계정 이름을 입력해주세요: SIT 고정자본형성
추가하실 금액을 입력해주세요: 1369.3
SIT 고정자본형성(40175.241511299995)항목에 + 1369.3 = 41544.5415113

승수분석을 완료하였습니다. 


,농림어업,광업,제조업,도매 및 소매업,"전기, 가스, 증기 및 수도사업",건설업,운수업,숙박 및 음식점업,정보 및 통신업,금융 및 보험업,...,기업,정부,고정자본형성,SIT 고정자본형성,간접세,법인세,소득세,관세,수출,수입
구분,,,,,,,,,,,,,,,,,,,,,
농림어업,4402.421522,4.597067,34163.317487,52.980928,12.351729,370.568449,23.570263,9908.930908,21.602472,40.414910,...,0.000000,0.000000,361.680433,0.000000,0.000,0.000,0.000,0.000,1145.805084,0.000
광업,13.560802,52.117666,86456.156387,0.000000,13537.450531,5209.311382,0.000000,53.333028,0.000000,0.000000,...,0.000000,0.000000,41951.167101,0.000000,0.000,0.000,0.000,0.000,2683.046883,0.000
제조업,16907.093755,650.710519,587991.884193,11766.655597,13543.578258,99317.190649,34709.806303,53982.496425,5830.919698,4069.237800,...,0.000000,0.000000,144172.854325,0.000000,0.000,0.000,0.000,0.000,618894.431784,0.000
도매 및 소매업,1435.566746,34.545196,40798.567041,4581.055717,783.376116,5290.400512,1826.012188,8853.929720,5259.463034,609.830641,...,0.000000,0.000000,8316.871834,0.000000,0.000,0.000,0.000,0.000,22039.934364,0.000
"전기, 가스, 증기 및 수도사업",896.716876,105.019837,36869.068842,2449.222314,17522.676861,1066.781568,2645.840227,5282.286752,1287.578309,1458.603658,...,0.000000,987.496720,0.000000,0.000000,0.000,0.000,0.000,0.000,1065.267207,0.000
건설업,92.834664,18.654654,1358.512287,221.609735,677.393582,102.451726,254.329095,400.850403,273.432802,108.729738,...,0.000000,0.000000,257479.498528,0.000000,0.000,0.000,0.000,0.000,214.473878,0.000
운수업,735.357017,610.795827,45886.016613,13390.582083,1930.339189,2476.076440,10734.751068,1181.161558,1813.539430,874.146357,...,0.000000,0.000000,593.336008,0.000000,0.000,0.000,0.000,0.000,33744.537905,0.000
숙박 및 음식점업,574.382136,95.109466,11380.304271,7710.816742,1012.662664,1162.402548,2140.424783,1911.383217,1951.015190,4787.071084,...,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.000,10938.889066,0.000
정보 및 통신업,195.013629,15.376793,3866.397877,6089.506525,391.744294,409.045647,1498.926734,1357.112477,11147.708471,6913.614440,...,0.000000,0.000000,17022.023814,0.000000,0.000,0.000,0.000,0.000,8098.051081,0.000


In [9]:
data.to_excel('SCN1_ㅇㄴ.xlsx')